In [5]:
import numpy as np
import pandas as pd


In [10]:
df = pd.read_csv("20210311_v0.2-FSI.csv")

df.describe()


,Entity Registration Date,Entity Profile UEN,Entity Name,Primary Section Description
count,6613,6613,6613,6613
unique,346,6413,6606,1
top,2020-03-13,202024182H,MRJS INCORPORATED PTE. LTD.,FINANCIAL AND INSURANCE ACTIVITIES
freq,62,3,2,6613
